In [1]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import random
import timeit
import numpy as np
import sys, os
sys.path.append('..')
from datetime import datetime
from citipy import citipy
from config import weather_api_key

In [2]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs


In [3]:
coordinates = list(lat_lngs)
for coordinate in coordinates:
    print(citipy.nearest_city(coordinate[0], coordinate[1]).city_name,
          citipy.nearest_city(coordinate[0], coordinate[1]).country_code)

ushuaia ar
kavieng pg
grand river south east mu
mataura pf
kahului us
kuching my
sentyabrskiy ru
mar del plata ar
port alfred za
urusha ru
san cristobal ec
lebu cl
barentsburg sj
bluff nz
nikolskoye ru
punta arenas cl
castro cl
fortuna us
pevek ru
ushuaia ar
chokurdakh ru
mount gambier au
new norfolk au
ilulissat gl
albany au
bluff nz
port elizabeth za
new norfolk au
sentyabrskiy ru
bredasdorp za
yellowknife ca
comodoro rivadavia ar
cape town za
ostersund se
port alfred za
kempele fi
san juan ar
butaritari ki
mitu co
taolanaro mg
lagoa pt
albany au
hasaki jp
burica pa
manggar id
grand centre ca
qaanaaq gl
nanortalik gl
vallenar cl
dudinka ru
barrow us
busselton au
ancud cl
avera pf
iwanai jp
mount gambier au
brae gb
tasiilaq gl
pangnirtung ca
ushuaia ar
albany au
punta arenas cl
vaini to
harda in
hobart au
taolanaro mg
lebu cl
illoqqortoormiut gl
chuy uy
port blair in
thompson ca
sumbe ao
atuona pf
ushuaia ar
atuona pf
barrow us
atuona pf
coquimbo cl
port alfred za
vila franca do campo

In [4]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

722

In [5]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Metric&APPID=" + weather_api_key


In [6]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1

    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_description= city_weather['weather'][0]["description"]

        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description": city_description
                          })

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | ushuaia
Processing Record 2 of Set 1 | kavieng
Processing Record 3 of Set 1 | grand river south east
City not found. Skipping...
Processing Record 4 of Set 1 | mataura
Processing Record 5 of Set 1 | kahului
Processing Record 6 of Set 1 | kuching
Processing Record 7 of Set 1 | sentyabrskiy
City not found. Skipping...
Processing Record 8 of Set 1 | mar del plata
Processing Record 9 of Set 1 | port alfred
Processing Record 10 of Set 1 | urusha
Processing Record 11 of Set 1 | san cristobal
Processing Record 12 of Set 1 | lebu
Processing Record 13 of Set 1 | barentsburg
City not found. Skipping...
Processing Record 14 of Set 1 | bluff
Processing Record 15 of Set 1 | nikolskoye
Processing Record 16 of Set 1 | punta arenas
Processing Record 17 of Set 1 | castro
Processing Record 18 of Set 1 | fortuna
Processing Record 19 of Set 1 | pevek
Processing Record 20 of Set 1 | chokurdakh
Processing Record 21 of

In [7]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.to_csv("WeatherPy_Database.csv")